
# 🌳 Karar Ağacı & Rastgele Orman (Random Forest)

# 1. Karar Ağacı

Karar Ağaçları, tahmine dayalı modelleme makine öğrenimi için önemli bir algoritma türüdür.

Klasik karar ağacı algoritmaları onlarca yıldır kullanılmaktadır ve rastgele orman gibi modern varyasyonları mevcut en güçlü teknikler arasındadır.

# 2. Random Forest
**Random Forest**, en popüler ve en güçlü makine öğrenme algoritmalarından biridir. **Bootstrap Aggregation veya bagging adı verilen bir tür topluluk makine öğrenimi algoritmasıdır.**


# 3. Karar Ağacı & Random Forest Uygulaması


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part1/Day7-DecisionTree/notebooks"
ROOT_DIR = "https://raw.githubusercontent.com/yapay-ogrenme/casgem-eu-project-training-on-data-mining/main/PART1/Day7-DecisionTree/notebooks/"

DATASET_PATH = ROOT_DIR + "/datasets/"

In [ ]:
df = pd.read_csv(DATASET_PATH + "WA_Fn-UseC_-HR-Employee-Attrition.csv")
df.head()

# Keşifsel Veri Analizi

In [ ]:
sns.countplot(x='Attrition', data=df)

In [ ]:
df.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis="columns", inplace=True)

categorical_col = []
for column in df.columns:
    if df[column].dtype == object and len(df[column].unique()) <= 50:
        categorical_col.append(column)
        
df['Attrition'] = df.Attrition.astype("category").cat.codes

# Veri İşleme

In [ ]:
categorical_col.remove('Attrition')

In [ ]:
# Transform categorical data into dummies
# categorical_col.remove("Attrition")
# data = pd.get_dummies(df, columns=categorical_col)
# data.info()
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
for column in categorical_col:
    df[column] = label.fit_transform(df[column])

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Attrition', axis=1)
y = df.Attrition

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Ağaç & Random Forest algoritmalarının uygulanması

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

## 1. Karar Ağacı ile Sınıflandıma

**Decision Tree parametreleri:**

- `criterion`: Bir bölünmenin kalitesini ölçme işlevi. Desteklenen kriterler Gini safsızlığı (Gini impurity) için "`gini`" ve bilgi kazancı için "`entropy`"dir.
***
- `splitter`: Her düğümde bölmeyi seçmek için kullanılan strateji. Desteklenen stratejiler, en iyi bölmeyi seçmek için "`best`" ve en iyi rastgele bölmeyi seçmek için "`random`"dir.
***
- `max_depth`: Ağacın maksimum derinliği. "`None`" ise, tüm yapraklar saf olana kadar veya tüm yapraklar "`min_samples_split`" örneklerinden daha azını içerene kadar düğümler genişletilir.
***
- `min_samples_split`: Bir dahili düğümü bölmek için gereken minimum örnek sayısı.
***
- `min_samples_leaf`: Bir yaprak düğümde olması gereken minimum örneklem sayısı. Herhangi bir derinlikteki bir bölünme noktası, yalnızca sol ve sağ dalların her birinde en az ``min_samples_leaf`` eğitim örneği bırakırsa dikkate alınacaktır.
***
- `max_features`: En iyi bölünmeyi ararken göz önünde bulundurulması gereken özelliklerin sayısı.
***
- `max_leaf_nodes`: En iyi şekilde "max_leaf_nodes" ile bir ağaç büyütmek anlamına gelir. En iyi düğümler, kirliliğin göreceli olarak azalması olarak tanımlanır. "None" ise, sınırsız sayıda yaprak düğümü anlamına gelir.
***
- `min_impurity_decrease`: Bir bölünme, ancak bu değerden daha büyük veya buna eşit değerde safsızlıkta bir azalmaya neden olursa, bir düğümün bölünmesi sağlanacaktır.
***
- `min_impurity_split`: Ağaç büyümesini erken durdurma eşiği (early stopping). Bir düğüm, safsızlık eşiğinin üzerindeyse bölünür, aksi takdirde bir yaprak olarak kalır.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

print_score(tree_clf, X_train, y_train, X_test, y_test, train=True)
print_score(tree_clf, X_train, y_train, X_test, y_test, train=False)

## 2. Karar Ağacı Hiperparametre Ayarlaması

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {
    "criterion":("gini", "entropy"), 
    "splitter":("best", "random"), 
    "max_depth":(list(range(1, 20))), 
    "min_samples_split":[2, 3, 4], 
    "min_samples_leaf":list(range(1, 20)), 
}


tree_clf = DecisionTreeClassifier(random_state=42)
tree_cv = GridSearchCV(tree_clf, params, scoring="accuracy", n_jobs=-1, verbose=1, cv=3)
tree_cv.fit(X_train, y_train)
best_params = tree_cv.best_params_
print(f"Best paramters: {best_params})")

tree_clf = DecisionTreeClassifier(**best_params)
tree_clf.fit(X_train, y_train)
print_score(tree_clf, X_train, y_train, X_test, y_test, train=True)
print_score(tree_clf, X_train, y_train, X_test, y_test, train=False)

### Ağacı görselleştirme

In [ ]:
from IPython.display import Image
from six import StringIO
from sklearn.tree import export_graphviz
import pydot

features = list(df.columns)
features.remove("Attrition")

In [ ]:
dot_data = StringIO()
export_graphviz(tree_clf, out_file=dot_data, feature_names=features, filled=True)
graph = pydot.graph_from_dot_data(dot_data.getvalue())
Image(graph[0].create_png())


# 3. Rastgele Orman (Random Forest)

Rastgele orman, veri kümesinin çeşitli alt örneklerinde bir dizi karar ağacı sınıflandırıcısına uyan bir meta tahmin edicidir ve tahmin doğruluğunu iyileştirmek ve overfittingi kontrol etmek için ortalamayı kullanır.

**Random forest algoritmasının parametreleri:**

- `n_estimators`: Ormandaki ağaç sayısı.
*** 
- `criterion`: Bir bölünmenin kalitesini ölçme işlevi. Desteklenen kriterler Gini safsızlığı (Gini impurity) için "`gini`" ve bilgi kazancı için "`entropy`"dir.
***
- `max_depth`
***
- `min_samples_split`
***
- `min_samples_leaf`
***
- `max_features`
***
- `max_leaf_nodes`
***
- `min_impurity_decrease`
***
- `min_impurity_split`
***
- `bootstrap`: Ağaç oluştururken önyükleme örneklerinin kullanılıp kullanılmadığı. "False" ise, her ağacı oluşturmak için tüm veri kümesi kullanılır.
***
- `oob_score`: Genelleme doğruluğunu tahmin etmek için torba dışı örneklerin kullanılıp kullanılmayacağı.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(X_train, y_train)

print_score(rf_clf, X_train, y_train, X_test, y_test, train=True)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=False)

## 4. Random Forest hiperparametre ayarlaması




### a) Randomized Search Cross Validation

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators, 'max_features': max_features,
               'max_depth': max_depth, 'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}

rf_clf = RandomForestClassifier(random_state=42)

rf_cv = RandomizedSearchCV(estimator=rf_clf, scoring='f1',param_distributions=random_grid, n_iter=100, cv=3, 
                               verbose=2, random_state=42, n_jobs=-1)

rf_cv.fit(X_train, y_train)
rf_best_params = rf_cv.best_params_
print(f"Best paramters: {rf_best_params})")

rf_clf = RandomForestClassifier(**rf_best_params)
rf_clf.fit(X_train, y_train)

print_score(rf_clf, X_train, y_train, X_test, y_test, train=True)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=False)

`Random search`, her bir hiperparametre için aralığı daraltmamıza izin verir. Artık aramamızı nereye yoğunlaştıracağımızı bildiğimize göre, denenecek her hiperparametre kombinasyonunu açıkça belirtebiliriz. Bunu, bir dağılımdan rastgele örnekleme yapmak yerine tanımladığımız tüm kombinasyonları değerlendiren bir yöntem olan `GridSearchCV` ile yapıyoruz.


### b) Grid Search Cross Validation

In [ ]:
n_estimators = [100, 500, 1000, 1500]
max_features = ['auto', 'sqrt']
max_depth = [2, 3, 5]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4, 10]
bootstrap = [True, False]

params_grid = {'n_estimators': n_estimators, 'max_features': max_features,
               'max_depth': max_depth, 'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}

rf_clf = RandomForestClassifier(random_state=42)

rf_cv = GridSearchCV(rf_clf, params_grid, scoring="f1", cv=3, verbose=2, n_jobs=-1)


rf_cv.fit(X_train, y_train)
best_params = rf_cv.best_params_
print(f"Best parameters: {best_params}")

rf_clf = RandomForestClassifier(**best_params)
rf_clf.fit(X_train, y_train)

print_score(rf_clf, X_train, y_train, X_test, y_test, train=True)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=False)